In [44]:
import pandas as pd
from typing import List
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer

In [45]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sabkr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sabkr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sabkr\AppData\Roaming\nltk_data...


True

In [3]:
ventas = pd.read_csv('./ventas.csv')
productos = pd.read_csv('./productos.csv')
clientes = pd.read_csv('./clientes.csv')
categorias = pd.read_csv('./categorias.csv')

A continuación, procedemos con el primer modelo solicitado por nuestro cliente, un sistema de recomendación basado en compras anteriores de clientes que no han vuelto a comprar.

In [9]:
clientes_unicos = ventas.groupby('email')['fecha'].max().reset_index()

In [10]:
clientes_unicos = clientes_unicos[clientes_unicos['fecha'] < '2024-06-01']

In [12]:
compras_pasadas = ventas[ventas['email'].isin(clientes_unicos['email'])]

In [15]:
stop_words = set(stopwords.words('english'))

In [16]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower()) 
    tokens = [word for word in tokens if word.isalpha()] 
    tokens = [word for word in tokens if word not in stop_words] 
    return ' '.join(tokens)

In [18]:
productos['descripcion_preprocesada'] = productos['nombre'].apply(preprocess_text)

In [21]:
vectorizer = TfidfVectorizer() #Vectorización
tfidf_matrix = vectorizer.fit_transform(productos['descripcion_preprocesada']) 

In [22]:
similaridades = cosine_similarity(tfidf_matrix) #Calcular la similitud del coseno entre productos

In [23]:
df_similaridades = pd.DataFrame(similaridades, index=productos['id_producto'], columns=productos['id_producto']) #DF de similitudes

In [27]:
def recomendar_productos(cliente_email, num_recomendaciones=5): #Función para obtener los productos recomendado para X cliente
    productos_comprados = compras_pasadas[compras_pasadas['email'] == cliente_email]['id_producto'].values #Obtener productos que el cliente ya compró
    recomendaciones = pd.Series(dtype=float) #Calcular similitudes con los productos que NO compró
    for producto in productos_comprados:
        recomendaciones = recomendaciones.add(df_similaridades[producto], fill_value=0)
        recomendaciones = recomendaciones.drop(productos_comprados).sort_values(ascending=False).head(num_recomendaciones) #Eliminar los productos que ya compró y ordenar por similitud
        return productos.loc[productos['id_producto'].isin(recomendaciones.index), ['nombre', 'precio']]
    

In [29]:
recomendar_productos('agustinalopez@gmail.com')

,nombre,precio
6,Gothic Lace Dress,42.99
64,Floral Print Dress,25.50
74,Cute Summer Dress,9.99
109,Y2K Sequin Dress,39.99
111,Black Gothic Dress,28.99


In [31]:
recomendar_productos('agustindiaz@gmail.com')

,nombre,precio
0,Kuromi Y2K Top,14.99
147,Kuromi Plush Doll,24.99
155,Chococat Wallet,29.99
156,Kuromi Crossbody Bag,35.99
170,Kuromi Sweatshirt,39.99


A continuación, implementaremos el sistema de recomendación para clientes nuevos, basándonos en los productos más populares de la tienda:

In [38]:
#Comenzamos por identificar los productos más vendidos
productos_populares = ventas.groupby('id_producto')['id_venta'].count().reset_index()
productos_populares = productos_populares.rename(columns={'id_venta': 'ventas_totales'})
productos_populares = productos_populares.sort_values(by='ventas_totales', ascending=False)

In [39]:
productos_populares = productos_populares.merge(productos, on='id_producto', how='left') #Unimos esta información al dataset original

In [58]:
def recomendar_productos_populares(num_recomendaciones=5): #Función para recomendar los productos más populares
    recomendaciones = productos_populares.head(num_recomendaciones)
    return recomendaciones[['nombre', 'precio', 'ventas_totales']]

In [59]:
print(recomendar_productos_populares(5))

                nombre  precio  ventas_totales
0  Gothic Lolita Dress  109.99              11
1    Kuromi Sweatshirt   39.99               7
2    Sanrio Phone Case   16.99               7
3       Kuromi Y2K Top   14.99               6
4   Black Combat Boots  120.00               6


Con esto, Noir ya tiene una noción de en qué productos le conviene seguir invirtiendo, y qué productos quizás sea mejor descontinuar, ya que generan poca o nula ganancia. Además, los clientes nuevos y los clientes perdidos, tienen mayor facilidad para comprar gracias a los sistemas de recomendación.